In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
from scipy.stats import norm,multivariate_normal

import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_svmlight_file
import torch
from torchvision import transforms
from sklearn.preprocessing import StandardScaler

In [245]:
# logging_index
# eval_index

# logging_policy 1
# eval_policy 1
# logging_xshift 1
# eval_xshift 1
# first_component 1
# logging_xy 1
# eval_xy 1
# logging_reward 1
# eval_reward 1
# logging_label1
# eval_reward 1

data = 'letter'

In [246]:
# box 1

# xy
# label
# fc
if data == 'opt':
    data = pd.read_csv('optdigits.csv',header = None)
    from sklearn.utils import shuffle
    data = shuffle(data)


    label = np.array(data[64]) - 1
    xy = np.array(data.drop([64],axis = 1))
    n_class = 10
    

elif data == 'satimage':
    data, labels = load_svmlight_file('satimage.scale')
    data = data.toarray()
    labels = np.array(labels,np.int64)
    index = np.random.permutation(len(data))
    xy = data[index]
    label = labels[index]
    label = label - 1
    n_class = max(label) + 1
    
elif data == 'veh':
    data, labels = load_svmlight_file('vehicle.scale')
    data = data.toarray()
    labels = np.array(labels,np.int64)
    index = np.random.permutation(len(data))
    xy = data[index]
    label = labels[index]
    label = label - 1
    n_class = max(label) + 1
    
elif data == 'mnist':
    path = 'training.pt'
    features_, labels = torch.load(path)
    size = 28
    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ])

    features = torch.zeros((len(features_), size, size))
    for i in range(len(features_)):
        features[i] = trans(features_[i])

    features = features.numpy().reshape(-1, size * size)
    labels = labels.numpy()

    xy = features
    label = labels
    n_class = 10

elif data == 'pen':
    data = pd.read_csv('pendigits.tra',sep = ',',header = None)
    data = shuffle(data)


    label = np.array(data[16])
    xy = np.array(data.drop([16],axis = 1))
    xy = StandardScaler().fit(xy).transform(xy)
    n_class = 10

elif data == 'letter':
    data = pd.read_csv('letter.csv')
    data = shuffle(data)


    label = np.array(data['0'])
    xy = np.array(data.drop(['0'],axis = 1))
    xy = StandardScaler().fit(xy).transform(xy)
    n_class = 26
    
    
n_dim = xy.shape[1]


train_xy = xy[:int(len(xy)*0.7)]
test_xy = xy[int(len(xy)*0.7):]

train_label = label[:int(len(xy)*0.7)]
test_label = label[int(len(xy)*0.7):]




pca_model = PCA(n_components=1).fit(xy)
fc = pca_model.transform(xy)

train_fc = fc[:int(len(xy)*0.7)]
test_fc = fc[int(len(xy)*0.7):]

train_size = int(len(xy)*0.7)



In [217]:
# box 2

logging_a = 0.8
logging_b = 0.3

eval_a = 1
eval_b = 0.01

def get_distribution(fc,a,b,pca_model=None):
    m = np.min(fc)
    m_bar = np.mean(fc)
    mean = m + (m_bar-m)/a
    std = np.sqrt((m_bar - m)/b)
#     pdf = norm(loc = mean,scale = std).pdf(fc)
    pdf = np.ones(len(fc))
    return pdf/sum(pdf), mean, std








In [218]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_soften(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(train_xy[:int(train_size*0.1)], train_label[:int(train_size*0.1)])
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = alpha + u*beta
            else:
                tmp[0,t] = (1-alpha - u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(train_xy, train_label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [254]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_adv(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 1- alpha - u*beta
            else:
                tmp[0,t] = (alpha + u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [255]:
# sample data
# just sample index and everything can be refered from index.
def take_action(policy):
    action = random.choices([t for t in range(n_class)],policy)[0]
    prob = policy[action]
    return action, prob


def sample_logging_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    logging_index = np.zeros(size,np.int)
    logging_action = np.zeros(size,np.int)
    logging_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
#         t_index = random.choices(index_list,weights=pdf[:train_size])[0]
        t_index = i
        logging_index[i] = t_index
        
        action,_ = take_action(logging_policy[t_index])
        logging_action[i] = action
        
        logging_reward[i] = int(action == label[t_index])
    
    return logging_index, logging_action, logging_reward




def sample_eval_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    eval_index = np.zeros(size,np.int)
    eval_action = np.zeros(size,np.int)
    eval_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
        t_index = random.choices(index_list,weights=np.ones(len(xy)))[0]
        t_index = t_index  + train_size
        eval_index[i] = t_index  
        
        action,_ = take_action(logging_policy[t_index])
        eval_action[i] = action
        
        eval_reward[i] = int(action == label[t_index-train_size])
    
    return eval_index , eval_action, eval_reward




        



In [256]:
def p_xshift(train_x,test_x):
    from sklearn.linear_model import LogisticRegression
    X = np.concatenate((train_x,test_x),axis = 0)
#     X = np.sum(X,axis = 1,keepdims = True)
    Y = np.concatenate((np.ones(len(train_x)),np.zeros(len(test_x))),axis = 0)
    clf = LogisticRegression().fit(X,Y)
    return clf

def loggingpolicy(x,y):
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression().fit(x,y)
    return clf

In [257]:
def evaluate_ips(logging_index, logging_action, logging_reward,\
                 eval_index, eval_action, eval_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT):
    
    
    
    
#     GT = np.mean(eval_reward)
    ips_list = []
    snips_list = []
    ips_r_list = []
    
    ips_list_unknown = []
    snips_list_unknown = []
    ips_r_list_unknown = []
    
    
    
    for k in range(1):
#         total_reward = 0
#         for i in range(len(eval_index)):
#             action, prob = take_action(eval_policy[eval_index[i]])
#             if action == label[eval_index[i]]:
#                 total_reward += 1
#         GT = total_reward/len(eval_index)

        
    #     known:

        p_w = my_bound(eval_policy[eval_index, eval_action]/logging_policy[eval_index, eval_action])
#         x_w = pdf_eval[logging_index]/pdf[logging_index]
        x_w = 1
        
        

        ips = p_w*eval_reward
        ips_loss = (np.mean(ips) - GT)**2
        ips_list.append(ips_loss)

        snips = p_w*eval_reward/np.mean(p_w)
        snips_loss = (np.mean(snips) - GT)**2
        snips_list.append(snips_loss)

        ips_r = x_w*p_w*eval_reward
        ips_r_loss = (np.mean(ips_r) - GT)**2
        ips_r_list.append(ips_r_loss)




    #     unknown:
        clf_xshift = p_xshift(xy[logging_index],xy[eval_index])
        clf_logging = loggingpolicy(xy[logging_index],logging_action)


        logging_predicted = clf_logging.predict_proba(xy[eval_index])[np.arange(len(eval_index)),np.array(eval_action,np.int)]
        p_w = eval_policy[eval_reward, eval_action]/logging_predicted

        xshift_logging = clf_xshift.predict_proba(xy[eval_reward])
        x_w = xshift_logging[:,1]/xshift_logging[:,0]



        ips = p_w*eval_reward
        ips_loss_unknown = (np.mean(ips) - GT)**2
        ips_list_unknown.append(ips_loss_unknown)

        snips = p_w*eval_reward/np.mean(p_w)
        snips_loss_unknown = (np.mean(snips) - GT)**2
        snips_list_unknown.append(snips_loss_unknown)

        ips_r = x_w*p_w*eval_reward
        ips_r_loss_unknown = (np.mean(ips_r) - GT)**2
        ips_r_list_unknown.append(ips_r_loss_unknown)

    
    return np.mean(ips_list),np.mean(snips_list),np.mean(ips_r_list),\
                np.mean(ips_list_unknown),np.mean(snips_list_unknown),np.mean(ips_r_list_unknown),
    
    

    
    
    
    
    
    
    
    
    

In [262]:
from ope_v3 import *


class WEIGHT_DATA_SET(data.Dataset):
    '''
    dataset class with instance weight 
    '''
    def __init__(self, data,reward, weights):
        '''
        weights are same dimensional witi original data
        '''
        self.data = torch.tensor(data)
        self.reward = torch.tensor(reward)
        self.weights = torch.tensor(weights)

    def __getitem__(self, index):
        img = self.data[index] 
        target = self.reward[index]
        weight = self.weights[index]
        return img, target, weight

    def __len__(self):
        return len(self.data)

# r is xshift weight
# w is policy shift weight
def train_robust_regression(logging_index, logging_action, logging_reward,\
                 eval_index, eval_action, eval_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                            dm=False,xshift_known = True, policy_known = True,use_xshift = True):
#     print('train models with true density ratios')
    Myy_robust_list = []
    Myx_robust_list = []
    model_robust_list = []
    
    feature = xy[logging_index]
    eval_xshift = pdf_eval[logging_index]
    logging_xshift = pdf[logging_index]
    eval_policy = eval_policy[logging_index,logging_action]
    logging_policy = logging_policy[logging_index,logging_action]
    
    
    for i in range(n_class):
        ## generate training set for action i

        idx = (logging_action == i)


        data_i = feature[idx]
        reward_i = logging_reward[idx]
        eval_xshift_i = eval_xshift[idx]
        logging_xshift_i = logging_xshift[idx]
        eval_policy_i = eval_policy[idx]
        logging_policy_i = logging_policy[idx]

        train_size = len(data_i)

        if xshift_known:
            r = logging_xshift_i/eval_xshift_i
        if policy_known:
            w = logging_policy_i/eval_policy_i
        if use_xshift:
            w = r * w
        
        if dm:
            w = torch.ones(len(data_i))
        

        weight_st = my_bound(w)
        weighted_train = WEIGHT_DATA_SET(data_i,reward_i, w)

        train_model = Net()

        validate_size = int(0.1*train_size)

        validate_loader = data.DataLoader(data.Subset(weighted_train, range(0, validate_size)),
            batch_size=32, shuffle=True)
        # 10% validation set
        train_loader = data.DataLoader(data.Subset(weighted_train, range(validate_size, train_size)),
            batch_size=32, shuffle=True, )

        train_model, Myy, Myx, _, _, _ = train_validate_test(args,args.lr_robust, 100, "regression", 'cpu', 'False', train_model,
            train_loader, None , validate_loader, n_class , 0.000, d = n_dim,testflag = False)

        model_robust_list.append(train_model)
        Myy_robust_list.append(Myy)
        Myx_robust_list.append(Myx)
    return model_robust_list, Myy_robust_list, Myx_robust_list




In [263]:
def predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                        logging_action,\
                       xy,  pdf, pdf_eval, logging_policy, eval_policy,\
                       dm = False,xshift_known = True, policy_known = True,use_xshift = True):
    
    
    
    if xshift_known:
        r = pdf/pdf_eval
        
    if policy_known:
        w = logging_policy/eval_policy
#     print(pdf/pdf_eval)
#     print(logging_policy/eval_policy)
    if use_xshift:
        w = r*w
    if dm:
        w = torch.tensor([1])
    
    weight_st = my_bound(w)
    if len(xy.shape) == 2:
        output = model_robust_list[int(logging_action)](torch.tensor(xy))
    else:
        output = model_robust_list[int(logging_action)](torch.tensor(xy).unsqueeze(0))

    Myy = Myy_robust_list[int(logging_action)]
    Myx = Myx_robust_list[int(logging_action)]

    meanY, varY = ru.predict_regression(torch.tensor(weight_st), Myy, Myx, output, mean0, var0)
    return meanY.detach().numpy()
        
        
        

In [264]:
def evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            GT, model_robust_list, Myy_robust_list, Myx_robust_list,\
                                            dm = False,use_xshift = True):
    
#     model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
#                                             eval_index, eval_action, eval_reward,\
#                                             xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
#                                             dm=dm, use_xshift = use_xshift)
    
#     sample action 
    reg_loss = []
    triple_loss = []
    triple_r_loss = []
    sn = np.mean(eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action])
    
    tmp_pred = np.zeros((len(eval_index),n_class))
    for i in range(n_class):
        tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        i,xy[eval_index],  pdf[eval_index], pdf_eval[eval_index], \
                                        logging_policy[eval_index,i], eval_policy[eval_index,i],\
                                        dm = dm,use_xshift = use_xshift)
        tmp_eval = tmp_eval.reshape(-1,len(eval_index))
        tmp_pred[:,i] = tmp_eval
    
    
    regression_pred = np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
    triple = eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action] * (eval_reward - \
                                    tmp_pred[np.arange(len(eval_index)),eval_action]) + np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
    sn_triple = eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action]/sn * (eval_reward - \
                                    tmp_pred[np.arange(len(eval_index)),eval_action]) + np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
                                                                                        

  


    return (np.mean(regression_pred)-GT)**2,(np.mean(triple)-GT)**2,(np.mean(sn_triple)-GT)**2



In [265]:
def evaluate_robust(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            GT, model_robust_list, Myy_robust_list, Myx_robust_list,\
                                            dm = False,use_xshift = True):
    
#     model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
#                                             eval_index, eval_action, eval_reward,\
#                                             xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
#                                             dm=dm, use_xshift = use_xshift)
    
#     sample action 
    reg_loss = []
    triple_loss = []
    triple_r_loss = []
    sn = np.mean(eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action])
    for t in range(10):
        regression_list = []
        triple_r_list = []
        triple_list = []
        total_reward = 0
        for i in range(len(eval_index)):
            action, prob = take_action(eval_policy[eval_index[i]])
            rew = 0
            if action == label[eval_index[i]]:
                rew = 1
                total_reward += rew

            tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        action,xy[eval_index[i]],  pdf[eval_index[i]], pdf_eval[eval_index[i]], \
                                        logging_policy[eval_index[i],action], prob,\
                                        dm = dm,use_xshift = use_xshift)
            regression_list.append(tmp_eval)
            
            
            tmp_logging = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        eval_action[i],xy[eval_index[i]],  pdf[eval_index[i]], pdf_eval[eval_index[i]], \
                                        logging_policy[eval_index[i],eval_action[i]], eval_policy[eval_index[i],eval_action[i]],\
                                        dm = dm,use_xshift = use_xshift)
            
            

            #     triple
            w = eval_policy[eval_index[i],eval_action[i]]/logging_policy[eval_index[i],eval_action[i]]
            triple_list.append(w*(eval_reward[i] - tmp_logging) + tmp_eval)
            triple_r_list.append(w/sn*(eval_reward[i] - tmp_logging) + tmp_eval)
            
            
            
            rw = my_bound(pdf_eval[eval_index[i]]/pdf[eval_index[i]] * w)
            
            
            
            
#             w = my_bound(eval_policy[eval_index[i],logging_action[i]]/logging_policy[logging_index[i],logging_action[i]])

#             tmp = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
#                                         logging_action[i],xy[logging_index[i]], pdf[logging_index[i]], pdf_eval[logging_index[i]], \
#                                         logging_policy[logging_index[i],logging_action[i]], eval_policy[logging_index[i],logging_action[i]],\
#                                         dm = dm,use_xshift = use_xshift)
            
#             triple_r_list.append(rw*(logging_reward[i] - tmp) + tmp_eval)

#             triple_list.append(w*(logging_reward[i] - tmp) + tmp_eval)
 

        triple_r = np.mean(triple_r_list)
        triple = np.mean(triple_list)
        robust = np.mean(regression_list)
#         GT = total_reward/len(triple_r_list)
        reg_loss.append(robust)
        triple_loss.append(triple)
        triple_r_loss.append(triple_r)
    
    return (np.mean(reg_loss)-GT)**2,(np.mean(triple_loss)-GT)**2,(np.mean(triple_r_loss)-GT)**2



In [266]:
model_list = [
    'known ips',
    'known snips',
#     'known ips-r',
#     'unknown ips',
#     'unknown snips',
#     'unknown ips-r',
    'known dm',
    'known dmdr',
#     'known robust without xshift',
#     'known triple without xshift',
#     'known triple-r without xshift',
    'known robust',
    'known triple',
    'known sntriple'
#     'unknown dm',
#     'unknown dmdr',
#     'unknown dmdr-r',
#     'unknown robust without xshift',
#     'unknown triple without xshift',
#     'unknown triple-r without xshift',
#     'unknown robust',
#     'unknown triple',
#     'unknown triple-r'
]
len(model_list)

7

In [267]:
def estimate_GT_with_eval_policy(xy,label,eval_policy):
    eval_policy = eval_policy[train_size:]
    
    xy = xy[train_size:]
    
    label = label[train_size:]
    result = 0
    

    for i in range(len(xy)):
        result += eval_policy[i,label[i]]
    return result/len(eval_policy)
    
    

In [268]:
def return_dict_with_list(size_list):
    d = {}
    for i in range(len(size_list)):
        d[size_list[i]] = []
    return d

In [271]:
# ips
from tqdm import tqdm

# logging_a_list = [0.5,0.8,1.2]
# logging_b_list = [0.1,0.2,0.4]
size_list = [100]
train_size_list = [1000,5000,len(train_xy)]
# train_size_list = [1000]
logging_policy_list = [[0.7,0.2],[0.5,0.2],[0.4,0.2],[0.4,0.3],[0.1,0]]
# logging_policy_list = [[0.5,0.2],[0.1,0]]


cc = 0    
frame = {}
frame['model'] = model_list

for bandit_train_size in train_size_list:
    for logging_interval in logging_policy_list:
        cc += 1
        print(cc)
        logging_policy, eval_policy = generate_policy_soften(logging_interval[0],logging_interval[1])

        GT = estimate_GT_with_eval_policy(xy,label,eval_policy)


        ips = return_dict_with_list(size_list)
        snips = return_dict_with_list(size_list)
        ips_r =return_dict_with_list(size_list)

        ips_unknown =return_dict_with_list(size_list)
        snips_unknown =return_dict_with_list(size_list)
        ips_r_unknown =return_dict_with_list(size_list)

        robust = return_dict_with_list(size_list)
        triple_noxshift = return_dict_with_list(size_list)
        triple = return_dict_with_list(size_list)
        sntriple_noxshift =return_dict_with_list(size_list)

        dm = return_dict_with_list(size_list)
        dmdr_noxshift = return_dict_with_list(size_list)
        dmdr = return_dict_with_list(size_list)

        robust_without_xshift = return_dict_with_list(size_list)
        triple_without_xshift = return_dict_with_list(size_list)
        triple_r_without_xshift = return_dict_with_list(size_list)

        pdf, mean_logging, std_logging = get_distribution(fc,logging_a, logging_b)
        pdf_eval, mean_eval, std_eval = get_distribution(fc,eval_a, eval_b)


        for _ in tqdm(range(10)):

            logging_index, logging_action, logging_reward = \
                    sample_logging_data(train_xy, train_label, bandit_train_size, pdf, pdf_eval, logging_policy, eval_policy)

            eval_index, eval_action, eval_reward = \
                        sample_eval_data(test_xy, test_label, 100, pdf, pdf_eval, logging_policy, eval_policy)  

            model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            dm=False, use_xshift = True)
            model_dm_robust_list, Myy_dm_robust_list, Myx_dm_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            dm=True, use_xshift = True)


            for size in size_list:
                
                eval_index, eval_action, eval_reward = \
                        sample_eval_data(test_xy, test_label, size, pdf, pdf_eval, logging_policy, eval_policy)  
                res = evaluate_ips(logging_index, logging_action, logging_reward,\
                                 logging_index, logging_action, logging_reward,\
                                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT)
        
                ips[size].append(res[0])
                snips[size].append(res[1])


                ips_unknown[size].append(res[3])
                snips_unknown[size].append(res[4])



                res = evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                                            logging_index, logging_action, logging_reward,\
                                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT,\
                                                            model_robust_list, Myy_robust_list, Myx_robust_list,\
                                                            dm = False,use_xshift = False)
                robust[size].append(res[0])
                triple_noxshift[size].append(res[1])
                sntriple_noxshift[size].append(res[2])




                res = evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                    logging_index, logging_action, logging_reward,\
                                    xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT,\
                                    model_dm_robust_list, Myy_dm_robust_list, Myx_dm_robust_list,\
                                    dm = True,use_xshift = False)
                dm[size].append(res[0])
                dmdr_noxshift[size].append(res[1])
            for k in range(len(size_list)):
                size = size_list[k]
                policy_name = str(bandit_train_size) +'_'+ str(size_list[k]) + str(logging_interval)
                result_list = []
                
                

                result_list.append(np.sqrt(np.mean(ips[size])))
                result_list.append(np.sqrt(np.mean(snips[size])))

                result_list.append(np.sqrt(np.mean(dm[size])))
                result_list.append(np.sqrt(np.mean(dmdr_noxshift[size])))

                result_list.append(np.sqrt(np.mean(robust[size])))
                result_list.append(np.sqrt(np.mean(triple_noxshift[size])))
                result_list.append(np.sqrt(np.mean(sntriple_noxshift[size])))
                frame[policy_name] = result_list
frame = pd.DataFrame(frame).round(5)
print(frame)
# frame.to_csv('result2/newadv_noxshift.csv',index = False)


1


100%|███████████████████████████████████████████| 10/10 [00:22<00:00,  2.24s/it]


2


100%|███████████████████████████████████████████| 10/10 [00:22<00:00,  2.27s/it]


3


...

14


100%|███████████████████████████████████████████| 10/10 [02:48<00:00, 16.82s/it]


15


100%|███████████████████████████████████████████| 10/10 [02:47<00:00, 16.74s/it]

            model  1000_100[0.7, 0.2]  1000_100[0.5, 0.2]  1000_100[0.4, 0.2]  \
0       known ips             0.07821             0.05039             0.05531   
1     known snips             0.06596             0.06518             0.05052   
2        known dm             0.05400             0.07459             0.07258   
3      known dmdr             0.05353             0.04252             0.03936   
4    known robust             0.04282             0.03842             0.05721   
5    known triple             0.06117             0.04798             0.04066   
6  known sntriple             0.06182             0.05050             0.03961   

   1000_100[0.4, 0.3]  1000_100[0.1, 0]  5000_100[0.7, 0.2]  \
0             0.03782           0.08213             0.04210   
1             0.05598           0.05980             0.02687   
2             0.07786           0.32713             0.03687   
3             0.02727           0.08749             0.02282   
4             0.06158           0.29

In [272]:
frame

model  1000_100[0.7, 0.2]  1000_100[0.5, 0.2]  1000_100[0.4, 0.2]  \
0       known ips             0.07821             0.05039             0.05531   
1     known snips             0.06596             0.06518             0.05052   
2        known dm             0.05400             0.07459             0.07258   
3      known dmdr             0.05353             0.04252             0.03936   
4    known robust             0.04282             0.03842             0.05721   
5    known triple             0.06117             0.04798             0.04066   
6  known sntriple             0.06182             0.05050             0.03961   

   1000_100[0.4, 0.3]  1000_100[0.1, 0]  5000_100[0.7, 0.2]  \
0             0.03782           0.08213             0.04210   
1             0.05598           0.05980             0.02687   
2             0.07786           0.32713             0.03687   
3             0.02727           0.08749             0.02282   
4             0.06158           0.29005             0.03104   
5             0.02879           0.05809             0.02695   
6             0.03276           0.05740             0.02645   

   5000_100[0.5, 0.2]  5000_100[0.4, 0.2]  5000_100[0.4, 0.3]  \
0             0.04208             0.02294             0.03600   
1             0.03184             0.01818             0.02533   
2             0.07885             0.12032             0.12050   
3             0.02894             0.01263             0.02547   
4             0.07526             0.10969             0.10271   
5             0.03160             0.01479             0.02360   
6             0.03165             0.01470             0.02250   

   5000_100[0.1, 0]  14000_100[0.7, 0.2]  14000_100[0.5, 0.2]  \
0           0.04390              0.02404              0.02639   
1           0.02527              0.02735              0.01679   
2           0.34358              0.05587              0.10985   
3           0.02711              0.02228              0.01713   
4           0.28247              0.03719              0.07575   
5           0.02526              0.02518              0.01692   
6           0.01908              0.02599              0.01668   

   14000_100[0.4, 0.2]  14000_100[0.4, 0.3]  14000_100[0.1, 0]  
0              0.01499              0.02212            0.02236  
1              0.01199              0.01046            0.02116  
2              0.13143              0.13275            0.34387  
3              0.01174              0.01066            0.01570  
4              0.10967              0.09666            0.28366  
5              0.01095              0.01033            0.01690  
6              0.01189              0.00975            0.01740

In [ ]:
frame_len = frame.copy()

In [273]:
frame.to_csv('result2/letter_result1_same.csv',index = False)

In [404]:
tmp_res

[0.0006503108584310573,
 1.5071289965870986e-05,
 0.00014912624994976813,
 0.0001966217316565533,
 0.002403711023301239,
 0.0003668544547247813,
 0.0003379121079380731]